https://www.kaggle.com/sengzhaotoo/movielens-small?select=ratings.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ai_school/data/Movielens(small)/ratings.csv')
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ai_school/data/Movielens(small)/movies.csv')

pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 300)

movie_ratings = pd.merge(ratings, movies, on='movieId')
print(movie_ratings)

        userId  movieId  rating   timestamp                                 title                                genres
0            1       31     2.5  1260759144                Dangerous Minds (1995)                                 Drama
1            7       31     3.0   851868750                Dangerous Minds (1995)                                 Drama
2           31       31     4.0  1273541953                Dangerous Minds (1995)                                 Drama
3           32       31     4.0   834828440                Dangerous Minds (1995)                                 Drama
4           36       31     3.0   847057202                Dangerous Minds (1995)                                 Drama
...        ...      ...     ...         ...                                   ...                                   ...
99999      664    64997     2.5  1343761859              War of the Worlds (2005)                         Action|Sci-Fi
100000     664    72380     3.5  1344435

In [3]:
title_user = movie_ratings.pivot_table('rating', index='userId', columns='title')
title_user.fillna(0, inplace=True)
print(title_user)

title   "Great Performances" Cats (1998)  $9.99 (2008)  'Hellboy': The Seeds of Creation (2004)  ...  ¡Three Amigos! (1986)  À nous la liberté (Freedom for Us) (1931)  İtirazım Var (2014)
userId                                                                                           ...                                                                                       
1                                    0.0           0.0                                      0.0  ...                    0.0                                        0.0                  0.0
2                                    0.0           0.0                                      0.0  ...                    0.0                                        0.0                  0.0
3                                    0.0           0.0                                      0.0  ...                    0.0                                        0.0                  0.0
4                                    0.0           0.0      

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
user_based_collab = cosine_similarity(title_user, title_user)
print(user_based_collab)

[[1.         0.         0.         ... 0.06291708 0.         0.01746565]
 [0.         1.         0.12429498 ... 0.02413984 0.17059464 0.1131753 ]
 [0.         0.12429498 1.         ... 0.08098382 0.13660585 0.17019275]
 ...
 [0.06291708 0.02413984 0.08098382 ... 1.         0.04260878 0.08520194]
 [0.         0.17059464 0.13660585 ... 0.04260878 1.         0.22867673]
 [0.01746565 0.1131753  0.17019275 ... 0.08520194 0.22867673 1.        ]]


In [5]:
user_based_collab = pd.DataFrame(user_based_collab, index=title_user.index, columns=title_user.index)
print(user_based_collab)

userId       1         2         3    ...       669       670       671
userId                                ...                              
1       1.000000  0.000000  0.000000  ...  0.062917  0.000000  0.017466
2       0.000000  1.000000  0.124295  ...  0.024140  0.170595  0.113175
3       0.000000  0.124295  1.000000  ...  0.080984  0.136606  0.170193
4       0.074482  0.118821  0.081640  ...  0.104309  0.054512  0.211609
5       0.016818  0.103646  0.151531  ...  0.038358  0.062642  0.225086
...          ...       ...       ...  ...       ...       ...       ...
667     0.000000  0.425462  0.124562  ...  0.018416  0.153111  0.127995
668     0.000000  0.084646  0.124911  ...  0.000000  0.178017  0.135387
669     0.062917  0.024140  0.080984  ...  1.000000  0.042609  0.085202
670     0.000000  0.170595  0.136606  ...  0.042609  1.000000  0.228677
671     0.017466  0.113175  0.170193  ...  0.085202  0.228677  1.000000

[671 rows x 671 columns]


In [6]:
# 1번째 유저와 유사한 10명의 유저를 뽑는다
print(user_based_collab[1].sort_values(ascending=False)[:10])

userId
1      1.000000
325    0.371852
634    0.194093
341    0.162819
310    0.157524
207    0.152746
35     0.130585
195    0.122647
485    0.114021
130    0.112817
Name: 1, dtype: float64


In [7]:
# 가장 유사한 유저를 뽑아서 해당 유저가 본 영화 추천
user = user_based_collab[1].sort_values(ascending=False)[:10].index[1]
result = title_user.query(f"userId == {user}").sort_values(ascending=False, by=user, axis=1)
print(result)

title   Beverly Hills Cop (1984)  Dangerous Minds (1995)  Brady Bunch Movie, The (1995)  ...  Frogs for Snakes (1998)  Fritz the Cat (1972)  İtirazım Var (2014)
userId                                                                                   ...                                                                    
325                          4.5                     4.5                            4.0  ...                      0.0                   0.0                  0.0

[1 rows x 9064 columns]


In [8]:
# 1번 유저와 유사한 유저들 9명을 뽑아서, 그 유저들이 어떤 영화에 대해서 부여한 평점에
# 유사도만큼의 가중치를 부여해서 이걸 토대로 1번 유저가 부여할 평점을 계산/예측
# 가중치 -> 유저 9명 유사도의 합 중에서 해당 유저가 차지하는 유사도
user_index_list = user_based_collab[1].sort_values(ascending=False)[:10].index.tolist()
user_weight_list = user_based_collab[1].sort_values(ascending=False)[:10].tolist()
print(user_index_list)
print(user_weight_list)
print(sum(user_weight_list)-1)

[1, 325, 634, 341, 310, 207, 35, 195, 485, 130]
[1.0, 0.3718515795200445, 0.19409305170790575, 0.16281928881328767, 0.1575243302750048, 0.15274612900892096, 0.13058496348265256, 0.12264701454037472, 0.11402063453702121, 0.11281730419223501]
1.5191042960774475


In [9]:
# 유사한 유저들조차도 평점을 부여하지 않았다면
# 1번 유저가 부여할 평점을 예측할 수 없다...
movie_title = 'Dark Knight, The (2008)'
weighted_sum = []
for i in range(1, 10):
    weighted_sum.append(title_user[movie_title][user_index_list[i]] * user_weight_list[i]/(sum(user_weight_list)-1))
print(sum(weighted_sum))

0.0


# 아이템 기반

In [10]:
user_title = movie_ratings.pivot_table('rating', index='title', columns='userId')
user_title.fillna(0, inplace=True)
print(user_title)

userId                                     1    2    3    ...  669  670  671
title                                                     ...               
"Great Performances" Cats (1998)           0.0  0.0  0.0  ...  0.0  0.0  0.0
$9.99 (2008)                               0.0  0.0  0.0  ...  0.0  0.0  0.0
'Hellboy': The Seeds of Creation (2004)    0.0  0.0  0.0  ...  0.0  0.0  0.0
'Neath the Arizona Skies (1934)            0.0  0.0  0.0  ...  0.0  0.0  0.0
'Round Midnight (1986)                     0.0  0.0  0.0  ...  0.0  0.0  0.0
...                                        ...  ...  ...  ...  ...  ...  ...
xXx (2002)                                 0.0  0.0  0.0  ...  0.0  0.0  0.0
xXx: State of the Union (2005)             0.0  0.0  0.0  ...  0.0  0.0  0.0
¡Three Amigos! (1986)                      0.0  0.0  0.0  ...  0.0  0.0  0.0
À nous la liberté (Freedom for Us) (1931)  0.0  0.0  0.0  ...  0.0  0.0  0.0
İtirazım Var (2014)                        0.0  0.0  0.0  ...  0.0  0.0  0.0

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# 영화와 영화 간의 유사도
item_based_collab = cosine_similarity(user_title, user_title)
print(item_based_collab)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.05821787 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.05821787 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [12]:
item_based_collab = pd.DataFrame(item_based_collab, index=user_title.index, columns=user_title.index)
print(item_based_collab)

title                                      "Great Performances" Cats (1998)  $9.99 (2008)  'Hellboy': The Seeds of Creation (2004)  ...  ¡Three Amigos! (1986)  À nous la liberté (Freedom for Us) (1931)  İtirazım Var (2014)
title                                                                                                                               ...                                                                                       
"Great Performances" Cats (1998)                                   1.000000      0.000000                                      0.0  ...               0.000000                                        0.0               0.0000
$9.99 (2008)                                                       0.000000      1.000000                                      0.0  ...               0.058218                                        0.0               0.0000
'Hellboy': The Seeds of Creation (2004)                            0.000000      0.000000                   

In [13]:
print(item_based_collab['Avatar (2009)'].sort_values(ascending=False)[:10])

title
Avatar (2009)                    1.000000
Inception (2010)                 0.633687
Iron Man (2008)                  0.633037
Dark Knight, The (2008)          0.575307
District 9 (2009)                0.563332
Star Trek (2009)                 0.561453
Sherlock Holmes (2009)           0.560310
Up (2009)                        0.555886
Dark Knight Rises, The (2012)    0.548584
WALL·E (2008)                    0.535258
Name: Avatar (2009), dtype: float64


# 잠재 요인

In [14]:
print(user_title)

userId                                     1    2    3    ...  669  670  671
title                                                     ...               
"Great Performances" Cats (1998)           0.0  0.0  0.0  ...  0.0  0.0  0.0
$9.99 (2008)                               0.0  0.0  0.0  ...  0.0  0.0  0.0
'Hellboy': The Seeds of Creation (2004)    0.0  0.0  0.0  ...  0.0  0.0  0.0
'Neath the Arizona Skies (1934)            0.0  0.0  0.0  ...  0.0  0.0  0.0
'Round Midnight (1986)                     0.0  0.0  0.0  ...  0.0  0.0  0.0
...                                        ...  ...  ...  ...  ...  ...  ...
xXx (2002)                                 0.0  0.0  0.0  ...  0.0  0.0  0.0
xXx: State of the Union (2005)             0.0  0.0  0.0  ...  0.0  0.0  0.0
¡Three Amigos! (1986)                      0.0  0.0  0.0  ...  0.0  0.0  0.0
À nous la liberté (Freedom for Us) (1931)  0.0  0.0  0.0  ...  0.0  0.0  0.0
İtirazım Var (2014)                        0.0  0.0  0.0  ...  0.0  0.0  0.0

In [15]:
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12)
SVD_matrix = SVD.fit_transform(user_title)
print(SVD_matrix)
# 이번에는 코사인 유사도말고, 상관계수를 사용
corr = np.corrcoef(SVD_matrix)

[[ 1.22749115e-02  2.50643706e-03  1.55484607e-02 ... -1.97677834e-02
   1.75750943e-02 -7.04812520e-03]
 [ 4.23038342e-01 -6.90155544e-03 -4.06763357e-01 ... -5.59611249e-04
  -5.51151439e-01 -2.60451547e-01]
 [ 1.66327792e-01  1.08316705e-01  6.02898828e-02 ...  2.10853910e-01
   7.75587817e-02  2.61306193e-02]
 ...
 [ 8.51772031e+00 -3.31474827e+00  9.03947480e-01 ... -1.75642718e-01
   6.38754119e-01  2.13677986e+00]
 [ 3.19207059e-01  3.05393861e-01 -5.41609987e-01 ... -1.43656521e-02
  -5.71294496e-01 -9.63116091e-02]
 [ 1.04837793e-01  2.96773534e-01 -2.18210974e-01 ... -1.98814595e-01
   9.35087559e-02  1.01632914e-01]]


In [16]:
movie_title = title_user.columns.tolist()
print(movie_title)
movie_index = movie_title.index("Avengers, The (2012)")
print(movie_index)
top_index = np.where(corr[movie_index] > 0.9)
for i in top_index[0]:
    print(movie_title[i])

['"Great Performances" Cats (1998)', '$9.99 (2008)', "'Hellboy': The Seeds of Creation (2004)", "'Neath the Arizona Skies (1934)", "'Round Midnight (1986)", "'Salem's Lot (2004)", "'Til There Was You (1997)", "'burbs, The (1989)", "'night Mother (1986)", '(500) Days of Summer (2009)', '*batteries not included (1987)', '...And God Spoke (1993)', '...And Justice for All (1979)', '1-900 (06) (1994)', '10 (1979)', '10 Attitudes (2001)', '10 Cloverfield Lane (2016)', '10 Items or Less (2006)', '10 Things I Hate About You (1999)', '10 Years (2011)', '10,000 BC (2008)', '100 Girls (2000)', '100 Rifles (1969)', '101 Dalmatians (1996)', '101 Dalmatians (One Hundred and One Dalmatians) (1961)', '101 Reykjavik (101 Reykjavík) (2000)', '102 Dalmatians (2000)', '10th Kingdom, The (2000)', '11\'09"01 - September 11 (2002)', '11:14 (2003)', '12 Angry Men (1957)', '12 Angry Men (1997)', '12 Years a Slave (2013)', '127 Hours (2010)', '12:08 East of Bucharest (A fost sau n-a fost?) (2006)', '13 Assassin